# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
len(song_files)

71

In [6]:
song_files[:10]

['/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAPK12

In [7]:
filepath = '.'

In [8]:
df = pd.read_json(song_files[0], lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = tuple(df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']].astype(str).tolist())

song_data

('SOFSOCN12A8C143F5D',
 'Face the Ashes',
 'ARXR32B1187FB57099',
 '2007',
 '209.60608')

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
song_table_insert

'\nINSERT INTO songs ("song_id", "title", "artist_id", "year", "duration")      VALUES (%s, %s, %s, %s, %s)\n'

In [11]:
song_data

('SOFSOCN12A8C143F5D',
 'Face the Ashes',
 'ARXR32B1187FB57099',
 '2007',
 '209.60608')

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARXR32B1187FB57099,Gob,,NaN,NaN


In [14]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].iloc[0].values.tolist()
artist_data

['ARXR32B1187FB57099', 'Gob', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')
log_files

['/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-21-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-12-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-08-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-04-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-14-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-15-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-17-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-23-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-

In [17]:
filepath = '.'

In [18]:
# Create dataframe with all log records
df_log = pd.DataFrame()
for log_file in log_files:
    df_current = pd.read_json(log_file, lines=True)
    df_log = pd.concat([df_log, df_current])

In [19]:
df_log.shape

(8056, 18)

In [20]:
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,741,None,307,1542760054796,None,
1,None,Logged In,Theodore,M,1,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,741,None,200,1542760086796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
2,None,Logged Out,None,None,0,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761399796,None,
3,None,Logged Out,None,None,1,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761485796,None,
4,None,Logged Out,None,None,2,None,NaN,paid,None,PUT,Login,NaN,774,None,307,1542761486796,None,


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
df_log_time = df_log[df_log['page'] == 'NextSong'] 
df_log_time.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,1542761878796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,1542761921796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,1542762125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [22]:
df_log_time.ts.dtypes

dtype('int64')

In [23]:
df_log_time['ts_dt'] = pd.to_datetime(df_log_time.ts, unit='ms')

/home/gabriel/miniconda3/envs/data_eng_project1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_log_time.ts_dt.dtypes

dtype('<M8[ns]')

#### Create columns with datetime info

In [25]:
list_time_elements = ["hour",
                     "day",
                     "week",
                     "month",
                     "year",
                     "weekday"]

In [26]:
for e in list_time_elements:
    df_log_time['start_time'] = df_log_time['ts_dt']
    df_log_time[e] = getattr(df_log_time['ts_dt'].dt, e)

/home/gabriel/miniconda3/envs/data_eng_project1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/miniconda3/envs/data_eng_project1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
df_log_time.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,userAgent,userId,ts_dt,start_time,hour,day,week,month,year,weekday
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,...,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-21 00:57:58.796,2018-11-21 00:57:58.796,0,21,47,11,2018,2
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,...,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-21 00:58:41.796,2018-11-21 00:58:41.796,0,21,47,11,2018,2
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,...,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-21 01:02:05.796,2018-11-21 01:02:05.796,1,21,47,11,2018,2


In [28]:
column_labels = ['start_time'] + list_time_elements
column_labels

['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [29]:
time_df = df_log_time[column_labels]
time_df.tail()

,start_time,hour,day,week,month,year,weekday
150,2018-11-18 20:58:00.796,20,18,46,11,2018,6
151,2018-11-18 20:59:03.796,20,18,46,11,2018,6
152,2018-11-18 21:00:36.796,21,18,46,11,2018,6
153,2018-11-18 21:01:35.796,21,18,46,11,2018,6
156,2018-11-18 23:02:44.796,23,18,46,11,2018,6


In [30]:
time_df.dtypes

start_time    datetime64[ns]
hour                   int64
day                    int64
week                   int64
month                  int64
year                   int64
weekday                int64
dtype: object

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [31]:
time_df.iloc[:10,]

,start_time,hour,day,week,month,year,weekday
6,2018-11-21 00:57:58.796,0,21,47,11,2018,2
7,2018-11-21 00:58:41.796,0,21,47,11,2018,2
8,2018-11-21 01:02:05.796,1,21,47,11,2018,2
9,2018-11-21 01:03:13.796,1,21,47,11,2018,2
11,2018-11-21 01:05:29.796,1,21,47,11,2018,2
14,2018-11-21 01:09:37.796,1,21,47,11,2018,2
15,2018-11-21 01:13:22.796,1,21,47,11,2018,2
16,2018-11-21 01:16:52.796,1,21,47,11,2018,2
17,2018-11-21 01:20:24.796,1,21,47,11,2018,2
18,2018-11-21 01:24:03.796,1,21,47,11,2018,2


In [32]:
%%time
for i, row in time_df.iloc[:10,].iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

CPU times: user 12.9 ms, sys: 1.04 ms, total: 13.9 ms
Wall time: 27.1 ms


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
df_log.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,741,None,307,1542760054796,None,
1,None,Logged In,Theodore,M,1,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,741,None,200,1542760086796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52


In [34]:
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,,None,None,None,free
1,52,Theodore,Smith,M,free
2,,None,None,None,paid
3,,None,None,None,paid
4,,None,None,None,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [35]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [36]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

AttributeError: 'Series' object has no attribute 'song'

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.